In [1]:
#-*-encoding:utf-8-*-
import requests
from bs4  import BeautifulSoup
import random
import json
import time
import thread

def cn_4Asms(r,url,parma,header,proxy):
        r.post(url,data=parma,headers=header,proxies=proxy) 
        thread.exit_thread()
        
def cn_4ALogin(r,isProxy=0,proxyIp='7.175.57.56',proxyPort='8083'):
    proxy={}
    if isProxy==1:
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
    
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'zh-CN','Accept-Encoding':'gzip, deflate'}
    
    sendsms1='41608446240A6782F2A0F031426EDC066CF24674F3F0586A4D5FF056D75FF4AB'
    sendsms2='41608446240A6782F2A0F031426EDC066CF24674F3F0586AF1E3983438A09296068B27CDC69779838A7C55CD262073C96CF24674F3F0586A33D411E96614BA993A16DFEB2EF9D75B55FBFB72B350DD6D'
#                 '上面是1357Cook'
#     '       41608446240A6782F2A0F031426EDC066CF24674F3F0586AF1E3983438A09296B279671D1165AB116B295283AA6E45C8C790C1E0C84B5883'
    userInfo=sendsms2
    cn_name="shizhongxia"
    
    
    url='https://4a.gmcc.net'
    result1=r.get(url,headers=header,proxies=proxy,verify=False)
 
    #获取random_form
    soup1=BeautifulSoup(result1.text,'lxml')
    loginForm1=str(soup1.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')
#     print loginForm1
    #登陆界面
    url='https://4acasp.gmcc.net/jk.do?method=checkUserType&userId='+cn_name
    a=r.post(url,data={'method':'checkUserType','userId':cn_name},headers=header,proxies=proxy,verify=False)

    #-----应该返回：SUCCESSG|TshizhongxiaG|T------
#     print a.text
    url='https://4acasp.gmcc.net/loginForward.do?target=https://4a.gmcc.net/first.do?method=login&appCode=IAM000&sendsms='+sendsms1
    parma={'random_form':loginForm1,'loginPage':'/auth/nextlogin.jsp','smsNameText':cn_name,'smsName':cn_name}
    result2=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)
    soup2=BeautifulSoup(result2.text,'lxml')
    loginForm2=str(soup2.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')
    parma={'sendsms':sendsms2}
    s=r.post('https://4acasp.gmcc.net/jk.do?method=checkSMSUser',data=parma,headers=header,proxies=proxy,verify=False)
    
    url='https://4acasp.gmcc.net/jk.do?method=checkSMSJK&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp&userInfo='+userInfo
    r.post(url,headers=header,proxies=proxy,verify=False)
    url='https://4acasp.gmcc.net/sendSms.do?smsfrom=send&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp'

    #下面这一步获取短信验证码
    thread.start_new_thread(cn_4Asms,(r,url,parma,header,proxy))
    
    print u'已经发送短信验证码'
    smsinput=raw_input('验证码:')
#     这里应该要做检查
    print u'正在登陆，请稍后...'

    # 验证
    url='https://4acasp.gmcc.net/loginServlet.do?target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000'
    parma={'random_form':loginForm2,'loginPage':'/auth/nextUserLogin.jsp','userName':cn_name,'passWord':'','tokenName':cn_name,'isUsePin':'true','updatePin':'false','tokenpassword':'','pinCode':'','smsName':cn_name,'smsPwd':'1234qweR','dynamic_smsPwd':smsinput,'checkCodeTemp':'','figerName':'','challengeName':cn_name,'challegePassword':'','vpn':''}
    result=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)

    #信息同步
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    s=r.get('https://4a.gmcc.net/synAuthz.do?method=syn&rnd='+random2,headers=header,proxies=proxy,verify=False)
    #-----{success:true,code:'null',text:'null'}---------

    #登陆4A
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/main.jsp?rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/SystemmsgPortlet.lp?id=systemmsg&userID='+cn_name+'&resType=1',headers=header,proxies=proxy)
    random2='0.'+str(random.randint(10000000000000000,99999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/synAuthz.do?method=syn2AppServer&rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/page/resgroup/searchFrame.jsp',headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
    #get:https://4a.gmcc.net/page/resource/appres/appresourceQuery.do?method=listIni
    r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"pageSize":50,"pageNum":1,"totalRowNum":0,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}'},headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
           
    params2={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"totalRowNum":-1,"pageSize":50,"pageNum":1,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}','queryName':'','queryIP':'','isQuerySubTree':'1'}
    result2=r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data=params2,headers=header,proxies=proxy,verify=False)
    json2=json.loads(result2.text)
    print 'Json已经生成'
    return json2
           
def cn_json_load(json2,cn_system):
    cn_i=0
    for i in json2.get('data'):
        ss=i.get('name')
        if ss==cn_system:
            break
        cn_i=cn_i+1
    appResAccountList=json2.get('data')[cn_i].get('appResAccountList').split('|')[0][:-1]
    id_num=json2.get('data')[cn_i].get('id')
    return [appResAccountList,id_num]
           
           
def cn_ESOPLogin(r,json2,isProxy=0,proxyIp='7.175.57.56',proxyPort='8083'):
    if isProxy==0:
        proxy={}
    elif(isProxy==1):
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
    
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    #点击ESOP登陆
    cn_temp=cn_json_load(json2,u'广东移动NGESOP系统')
    appResAccountList=cn_temp[0]
    id_num=cn_temp[1]


    #同步数据
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/page/resource/resourceQuery.do?',data={'method':'isAuthDateValid','subaccName':'AGZGT0000829'},headers=header,proxies=proxy,verify=False)
    params2={'method':'checkPolicy4JK','resId':id_num,'resAccId':appResAccountList,'date':str(time.time()).replace('.','')+str(random.randint(1,9))}
    result2=r.post('https://4a.gmcc.net/page/jk/sso/appJKLogin.do',data=params2,headers=header,proxies=proxy,verify=False)
    params2={'method':'queryLoginJKStatus','resId':id_num,'resAccId':appResAccountList,'date':str(time.time()).replace('.','')+str(random.randint(1,9))}
    r.post('https://4a.gmcc.net/page/jk/sso/appJKLogin.do',data=params2,headers=header,proxies=proxy,verify=False)
    params2={'method':'wantGetMACAddress','resId':id_num}
    r.post('https://4a.gmcc.net/sso.do',data=params2,headers=header,proxies=proxy,verify=False)
    url2='https://4a.gmcc.net/sso.do?method=appssoData&accID='+appResAccountList+'&resID='+id_num+'&softname=webAppByIE&date='+str(time.time()).replace('.','')+str(random.randint(1,9))
    print '-----------测试是否登陆ESOP成功-----------'
    r.get(url2,headers=header,proxies=proxy,allow_redirects=True,verify=False)
    


In [2]:
r=requests.session()
header={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64; rv:44.0) Gecko/20100101 Firefox/44.0','Accept-Language':'en-US,en;q=0.5'}

json2=cn_4ALogin(r,0)


/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests

已经发送短信验证码


/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


验证码:737019
正在登陆，请稍后...


/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests

Json已经生成


In [3]:
cn_ESOPLogin(r,json2,0)

/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)
/home/chinafishz/opt/anaconda2/lib/python2.7/site-packages/requests

-----------测试是否登陆ESOP成功-----------


In [4]:
#test:查产品编码余额
import random

def cn_ESOP_CHAYUE(r,cn_servNumber):
    cn_random='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    s=r.get('http://esop.boss.gmcc.net/ngcustcare_fs/chargesrv/common/relateSubs.action?servNumber='+cn_servNumber+'&isSupportGrp=undefined&random='+cn_random)
    # print s.text
    soup=BeautifulSoup(s.text,'lxml')
    cn_subsId=soup.find(attrs={"name":"subsId"}).attrs['value']


    params2={'subsId':cn_subsId,'searchType':'servNumber','searchNumber':cn_servNumber,'balanceType':'CanUse'}
    s=r.post('http://esop.boss.gmcc.net/ngcustcare_fs/chargesrv/accountInfoQry/balanceQry/query.action',data=params2)
    # print s.text
    soup=BeautifulSoup(s.text,'lxml')
    temp=soup.select('.table_list03')[1].select('td')
    print cn_servNumber,''.join(temp[16].text).split()[0],''.join(temp[17].text).split()[0]
    
cn_Num=u'20168013476'

for i in cn_Num.split(';'):
     cn_ESOP_CHAYUE(r,i)

20168013476 可用预付金 0.00


In [4]:
#打开菜单
import random
from bs4 import BeautifulSoup
import time
import datetime
cn_groupId='2000100318'

cn_subsId='2006843085347'
cn_GROUPID='82000020636110'

def cn_cookies(cookies):
        cookies = dict()
        for i in r.cookies.get_dict('esop.boss.gmcc.net'):
            cookies[i]=r.cookies.get(i,'','esop.boss.gmcc.net')
        cookies['com.huawei.boss.CURRENT_MENUID']='ESOP_GRP_100010061038'
        cookies['com.huawei.boss.CURRENT_TAB']='ESOP_GRP_100010061038'
        return cookies

def cn_shanchengyuan(r,cn_SERVNUMBER):
    # 生成菜单

    s=r.get('http://esop.boss.gmcc.net/esop/dashboard/menu/ngcrm/toCommonMiddlePage.action?ngcrmMenuInfoBean.menuId=ESOP_GRP_100010061038&logId=customer')

    # 查询集团
    url='http://esop.boss.gmcc.net/ngcustcare_fs/group/commonquery/queryCustomerInfo.action?recType=GroupMemberMgrAdd'
    params={'groupOrMemeber':'group','groupId':cn_groupId,'grpMem.qryType':'SERVNUMBER','grpMem.inputVal':'','grpMem.startTime':'','grpMem.startTime_format':'yyyy-MM-dd HH:mm:ss','grpMem.endTime':'','grpMem.endTime_format':'yyyy-MM-dd HH:mm:ss'}
    r.post(url,data=params)
    url='http://esop.boss.gmcc.net/ngcustcare_fs/group/groupMemberMg/subsListInit.action'
    params={'groupOrMemeber':'group','groupId':cn_groupId,'grpMem.qryType':'ALL','grpMem.inputVal':'','grpMem.startTime':'','grpMem.startTime_format':'yyyy-MM-dd HH:mm:ss','grpMem.endTime':'','grpMem.endTime_format':'yyyy-MM-dd HH:mm:ss'}
    r.post(url,data=params)
    url='http://esop.boss.gmcc.net/ngcustcare_fs/group/commonquery/querySubscriberInfo.action?subsId='+cn_subsId
    r.post(url,data=params)
    
    #查memOid
    url='http://esop.boss.gmcc.net/ngcustcare_fs/group/groupMemberMg/query.action?grpMem.grpSubsId='+cn_subsId+'&grpMem.qryType=SERVNUMBER&grpMem.inputVal='+cn_SERVNUMBER+'&isAllowRec=1'
#     print url
    result=r.get(url) 
    soup=BeautifulSoup(result.text,'lxml')
    cn_memOid=soup.find(attrs={"title":u"删除"}).attrs['onclick'].split('#')[2]
#     print cn_memOid
    #点击删除
    cn_random='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    url="http://esop.boss.gmcc.net/ngcustcare_fs/group/groupMemberMg/preDelete.action?ngcrm_rnd="+cn_random+"&ngcrm_reserved1=1&subsId="+cn_subsId+"&servNumber="+cn_SERVNUMBER+"&memeberType=0&memOid="+cn_memOid+"&memShortNum=&productId=9105"
    r.get(url)  
    
    
    url="http://esop.boss.gmcc.net/ngcustcare_fs/group/groupMemberMg/delete.action?subsId="+cn_subsId+"&servNumber="+cn_SERVNUMBER+"&memeberType=0&memOid="+cn_memOid+"&memShortNum=&productId=9105&timeTag="+str(time.time()).replace(".","")+"0"+"&isDelFromCrm=&groupNo="+cn_groupId
    result=r.get(url) 
    #获取ESOP_GRP_100010061038_fee/calculate
    soup=BeautifulSoup(result.text,'lxml')
#     print soup
    cn_calculate=soup.find(attrs={"name":"_fee/calculate"}).attrs['value']
    
    #提交
    url="http://esop.boss.gmcc.net/ngcustcare_fs/common/fee/pay.action"
    params={"PreBussOrderId":"","withdrawGoods":"","payTypeBean.cashAmt":"","payTypeBean.chequeAmt":"","payTypeBean.bankId":"","payTypeBean.bankName":"","payTypeBean.chequeNum":"","payTypeBean.cardAmt":"","payTypeBean.posNum":"","payTypeBean.confirmPosNum":"","payTypeBean.weiXinPayAmt":"","payTypeBean.weiXinPayNum":"","payTypeBean.aliPayAmt":"","payTypeBean.aliPayPayNum":"","payTypeBean.acctCash":"","payTypeBean.scoreCash":"","payTypeBean.score":"","payTypeBean.scoreTransRule":"1/1","payTypeBean.m":"0","payTypeBean.mTransRule":"1/1","payTypeBean.separateAmt":"","payTypeBean.separateBankId":"GH","payTypeBean.separateBankName":u"工行","payTypeBean.separateBankAcct":"","payTypeBean.separateBankUserName":"","payTypeBean.separatePosNum":"","payTypeBean.limitCash":"","payTypeBean.unlimitCash":"","BARGAINFEEXML":"","CHANGEFEEBYBARGAINXML":"@260@263xml@232version@261@2341@2460@234@232encoding@261@234UTF@2458@234@263@262@260huawei@295call@262@260i@262common@247fee@247@242@260@247i@262@260e@262changeFeeByBargain@260@247e@262@260p@262@260m@262@260n@2620@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260m@262@260n@2621@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260@247p@262@260@247huawei@295call@262","UPDATEPAYTYPEBYPAGEDATAXML":"@260@263xml@232version@261@2341@2460@234@232encoding@261@234UTF@2458@234@263@262@260huawei@295call@262@260i@262common@247fee@247@242@260@247i@262@260e@262updatePayTypeByPageData@260@247e@262@260p@262@260m@262@260n@2620@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260@247p@262@260@247huawei@295call@262","reccustinfo_name":u"江*娟","reccustinfo_phone":"13602804206","reccustinfo_certificateType":"IdCard","reccustinfo_certificateNum":"********","reccustinfo_address":"","reccustinfo_note":"","invoicePrintMode":"","hiddenTokenName":"_fee/calculate","_fee/calculate":cn_calculate,"receiptNumber":"00000000","assembleInvoice":"0","invoiceNumber":"00000000","isForPaging":"OLD","changeEnumRecType":""}
    result=r.post(url,data=params)
    
    soup=BeautifulSoup(result.text,'lxml').select('#error_msg_title')[0].text
    
    return ''.join(soup).split()[0]

In [6]:
print cn_shanchengyuan(r,'13602893015')

业务办理成功!


In [5]:
### import time
from datetime import datetime


cn_list='13922114902;13922231978;13922116750;13925026057;13600079296;13602894590;13902494586;13902494592;13922117351;13922119070;13922112729;13802978524;13922114702;13802925589;13902210047;13926009702;13602845926;13802911995;13500028581;13922113356;13922117481;13922119012;13922119037;13922181700;13922404792;13925062701;13926400751;13600056860;13922119329;13903051457;13922113015;13922113791;13922114281;13925063785;13925166485;13926033388;13802535823;13922122281;13500222502;13826297038;13922112130;13922115423;13922736362;13922114018;13922118570;13922119361;13922115290;13922110401;13924100475;13802925593;13922113120;13922205493;13925015701;13826115250;13902226604;13602726475;13826104220;13922112705;13922115420;13922113406;13922114519;13902255190;13925092297;13609044821;13825168671;13922114712;13922119325;13922113712;13922210762;13926191549;13802539825;13926008888;13922182195;13922476269;13925066501;13602813775;13922292732;13609009530;13925025937;13802537066;13922268705;13602724098;13802977924;13600090050;13808872616;13922118300;13922115326;13922115391;13922116713;13602812774;13902265585;13922204331;13570022633;13600063417;13926018951;13560006084;13602726453;13602821704;13802925540;13826151161;13602884529;13922159738;13922241161;13924008544;13802972407;13922268709;13802988802;13924007044;13924267332;13922116451;13902212045;13924007613;13922112451;13822182026;13825182984;13826288636;13902260337;13602741972;13902494583;13922112531;13922113150;13922117125;13822204355;13922117502;13500002549;13922115329;13922245569;13922117631;13922143612;13602875146;13602884531;13822182048;13822187812;13922115798;13922117406;13922112426;13500024239;13925085501;13602893015;13802539626;13922209048;13925093101;13925195092;13824482222;13922116801;13925112369;13922119270;13925015573;13926272315;13602841141;13922116026;13922116350;13600044093;13922204213;13922102209;13922115125;13922118319;13922115830;13602869571;13922110370;13922112990;13922112609;13922113521;13922118205;13501512048;13922284873;13802908784;13902230343;13902291495;13922128130;13602804206;13609716110;13902229595;13826189988;13570003146;13826153220;13922119409;13602817969;13922209152;13925073201;13500226069;13503006685;13902291486;13922114891;13602734935;13802539159;13922128117;13500026923;13902494338;13922281687'
a=cn_list.split(';')
cn_a_count=len(a)
print cn_a_count
i=0
cn_time1=0
cn_time2=0

for cn_phonNnum in a:
    try:
        i=i+1
        print i,cn_phonNnum,cn_shanchengyuan(r,cn_phonNnum)
        time.sleep(1)x
    except:
        print '错误'
        continue

    

190
1 13922114902 业务办理成功!
2 13922231978 业务办理成功!
3 13922116750 业务办理成功!
4 13925026057 业务办理成功!
5 13600079296 业务办理成功!
6 13602894590 业务办理成功!
7 13902494586 业务办理成功!
8 13902494592 业务办理成功!
9 13922117351 业务办理成功!
10 13922119070 业务办理成功!
11 13922112729 业务办理成功!
12 13802978524 业务办理成功!
13 13922114702 业务办理成功!
14 13802925589 业务办理成功!
15 13902210047 业务办理成功!
16 13926009702 业务办理成功!
17 13602845926 业务办理成功!
18 13802911995 业务办理成功!
19 13500028581 业务办理成功!
20 13922113356 业务办理成功!
21 13922117481 业务办理成功!
22 13922119012 业务办理成功!
23 13922119037 业务办理成功!
24 13922181700 业务办理成功!
25 13922404792 业务办理成功!
26 13925062701 业务办理成功!
27 13926400751 业务办理成功!
28 13600056860 业务办理成功!
29 13922119329 业务办理成功!
30 13903051457 业务办理成功!
31 13922113015 业务办理成功!
32 13922113791 业务办理成功!
33 13922114281 业务办理成功!
34 13925063785 业务办理成功!
35 13925166485 业务办理成功!
36 13926033388 错误
37 13802535823 业务办理成功!
38 13922122281 业务办理成功!
39 13500222502 业务办理成功!
40 13826297038 业务办理成功!
41 13922112130 业务办理成功!
42 13922115423 业务办理成功!
43 13922736362 业务办理成功!
44 13922114018 业务办理成功